Sobre o teste

Os cinco grandes, modelo de cinco Fatores (MCF) fatores globais de personalidade (em inglês: big five) refere-se em psicologia aos cinco fatores da personalidade descritos pelo método lexical, ou seja, baseado em uma análise linguística) Abertura para a experiência (openness to experience); 2) Conscienciosidade (conscientiousness) 3) Extroversão (extraversion); 4) Neuroticismo ou Instabilidade Emocional (neuroticism); e 5) Amabilidade (agreeableness).

Os traços de personalidade Big Five são as medidas de personalidade cientificamente mais aceitos e mais comumente utilizados e têm sido extensivamente pesquisados. A técnica estatística utilizada para investigar esses traços por meio de questionário é o de Análise Fatorial, que é utilizada para reduzir uma quantidade grande de informações a um conjunto sintético e relevante.

O modelo Big Five é o resultado compreensivo, empírico e baseado em experimentos de pesquisas Identificar os traços e estrutura da personalidade humana é uma das metas principais da psicologia. Os cinco fatores foram descobertos e definidos por diversos pesquisadores independentes. Esses pesquisadores começaram pela investigação de traços de personalidade conhecidos e então analisaram centenas de medidas desses traços(em questionários de auto-avaliação, avaliações de terceiros e experimentos) para encontrar os fatores primordiais de personalidade.

A teoria identifica cinco fatores:

- ABERTURA PARA EXPERIÊNCIA (inventivo / curioso vs. consistente / cauteloso) (OPN)
- CONSCIÊNCIA (eficiente / organizado vs. extravagante / descuidado) (CSN)
- EXTROVERSÃO (saída / energética vs. solitária / reservada) (EXT)
- AMABILIDADE (amigável / compassivo vs. crítico / racional) (AGR)
- NEUROTICISMO (sensível / nervoso vs. resiliente / confiante) (EST)

In [ ]:
# Importando bibliotecas
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import os
from io import open
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
pd.options.display.max_columns = 150

In [ ]:
# Carregar a base de dados
dados = pd.read_csv('../input/big-five-personality-test/IPIP-FFM-data-8Nov2018/data-final.csv', sep='\t')

In [ ]:
# Visualizando a base de dados
dados.head()

In [ ]:
#Excluir dados não relevantes
dados.drop(dados.columns[50:110], axis=1, inplace=True)

In [ ]:
#Analisando dados estatísticos dos dados
dados.describe()

Pode-se verificar a presença de ruídos no sinal devido a ocorrência de valores mínimos igual a zero, onde o mínimo deveria ser de um. Os dados deverão ser tratados para que os mesmos não afetem o modelo criado posteriormente.

In [ ]:
#Limpando registros com valor iguais a zero
dados= dados[(dados > 0).all(axis=1)]
# Os dados agora só está levando em consideração aquelas respostas com valores superiores a zero(0)
dados.describe()

Pode-se observar que agora o mínimo em todas as respostas é igual a 1

Apesar da base de dados já deixar implicito que o teste foi feito com o intuito de separar as personalidades em 5 grupos, será aplicado adiante do método Elbow para apresentar a função que definirá a quantidade de clusters mais adequada para esse estudo.

In [ ]:
#Criando instancias para o método KMeans e o Visualizer
km = KMeans()
elbow = KElbowVisualizer(km, k=(2,10))

In [ ]:
#Gerando amostras
amostra = dados.sample(n=10000, random_state=1)

Foi decidido gerar uma amostra de todas as respostas ao invés de utilizar todo o universo de respostas para teste do modelo, devido a grande quantidade de respostas.  

In [ ]:
elbow.fit(amostra)
elbow.poof()

Como pode ser observado, o método Elbow retornou que a quantidade ideal de clusters é realmente 5. Então, os próximos passos do estudo serão feitos baseados nessa quantidade de 5 grupos.
Vale lembrar que o método aplicado foi para validação matemática da quantidade de clusters, mas que a mesma decisão poderia ter sido feita se fosse levada em consideração apenas a regra dos dados que já previa essa quantidade.

In [ ]:
# Agrupando os participantes
#Instanciando Kmeans
kmeans = KMeans(n_clusters=5)
kfit = kmeans.fit(dados)

In [ ]:
#Inserindo rótulos dos grupos
predicoes = kfit.labels_
dados['Clusters'] = predicoes

In [ ]:
dados.head()

In [ ]:
# Analisando os grupos
# Verificando a quantidade de observações em cada grupo
dados['Clusters'].value_counts()

In [ ]:
#Agrupando registros por grupos
dados.groupby('Clusters').mean()

In [ ]:
#Buscando padrões através das médias por cada tipo de pergunta
#Selecionando as coludas de cada grupo de perguntas
clist = list(dados)
ext = clist[0:10]
est = clist[10:20]
agr = clist[20:30]
cns = clist[30:40]
opn = clist[40:50]

In [ ]:
#Criando Data Frame para apresentação dos dados acima
mediagrupos = pd.DataFrame()
mediagrupos ['EXT'] = dados[ext].sum(axis=1)/10
mediagrupos ['EST'] = dados[est].sum(axis=1)/10
mediagrupos ['AGR'] = dados[agr].sum(axis=1)/10
mediagrupos ['CNS'] = dados[cns].sum(axis=1)/10
mediagrupos ['OPN'] = dados[opn].sum(axis=1)/10
mediagrupos ['Clusters'] = predicoes

In [ ]:
mediagrupos.groupby('Clusters').mean()

Para facilitar a visualização dos dados, é importante a geração de gráficos. Pensando nisso, será apresentado a seguir um gráfico de colunas com um gráfico de linhas representando as médias encontradas por grupo. 

In [ ]:
clusters = mediagrupos.groupby('Clusters').mean()
plt.figure(figsize=(22,3))
for i in range(0, 5):
    plt.subplot(1,5,i+1)
    plt.bar(clusters.columns, clusters.iloc[:, i], color='blue', alpha=0.2)
    plt.plot(clusters.columns, clusters.iloc[:, i], color='red')
    plt.title('Grupo' + str(i))
    plt.xticks(rotation=45)
    plt.ylim(0,4);

Para visualização de padrões dos agrupamentos de forma gráfica, será utilizado  método da análise de componentes principais(PCA)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pcafit = pca.fit_transform(dados)

dfpca = pd.DataFrame(data=pcafit, columns=['PCA1', 'PCA2'])
dfpca['Clusters'] = predicoes
dfpca.head()

In [ ]:
plt.figure(figsize=(10,10))
sb.scatterplot(data=dfpca, x='PCA1', y='PCA2', hue='Clusters', palette='Set1', alpha=0.9)
plt.title('Organização dos grupos após PCA');

Conclusão:
- O programa foi capaz de definir os 5 grupos para o agrupamento das pessoas de acordo com as respostas
- Os grupos 0,1,2 e 3 ficaram bem definidos
- O grupo 4 teve uma menor consistência no agrupamento
- Pessoas do grupo 2 e 4 possuem uma maior semelhança entre as respostas
- Dados mais precisos podem ser obtidos analisando mais a fundo as perguntas. Separando talvez em respostas Positivas e Negativas